In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
# 查看raw_data目录下所有文件
input_dir='./raw_data/'
files = os.listdir(input_dir)

# 创建列表保存读取的xlsx文件
data_list = []

# 读取excel文件
for file in files:
    data_list.append(pd.read_excel(input_dir + file))

In [3]:
# 合并所有文件
data = pd.concat(data_list)

# 重整列索引，非常重要!!!
data = data.reset_index()
data = data.drop("index",axis=1)

# 查看数据记录个数
print(len(data))

# 预览读入文件中的前5条记录
data.head(5)

52256


,描述,位置信息,区域,房屋信息,关注信息,总价,单价
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,3室2厅 | 86.4平米 | 东 | 简装 | 中楼层(共26层) | 2016年建 | 塔楼,135人关注 / 6个月以前发布,91.8万,单价10625元/平米
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,3室1厅 | 66.12平米 | 东 | 毛坯 | 高楼层(共33层) | 2017年建 |...,40人关注 / 2个月以前发布,128.5万,单价19435元/平米
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,3室1厅 | 73.59平米 | 南 | 精装 | 中楼层(共34层) | 2017年建 |...,58人关注 / 15天以前发布,153万,单价20791元/平米
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,2室1厅 | 75.4平米 | 东 北 | 简装 | 中楼层(共16层) | 2011年建 ...,36人关注 / 2个月以前发布,89万,单价11804元/平米
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,1室1厅 | 51.69平米 | 南 | 精装 | 高楼层(共15层) | 2007年建 | 板楼,38人关注 / 1个月以前发布,64.5万,单价12479元/平米


In [4]:
# 检索每条房屋信息记录中保存的信息个数
nrec = data.房屋信息.map(lambda x : len(x.split('|')))
nrec.value_counts()

7    39045
6    12669
8      370
4      122
5       50
Name: 房屋信息, dtype: int64

In [5]:
# 只保留有7条信息的记录
data = data[nrec == 7]
# 打印现有记录长度
len(data)

39045

In [6]:
# 拆分房屋信息所在列，并将拆分结果增至新标签列
data['户型'] = data.房屋信息.map(lambda x : x.split('|')[0])
data['面积'] = data.房屋信息.map(lambda x : x.split('|')[1])
data['朝向'] = data.房屋信息.map(lambda x : x.split('|')[2])
data['类型'] = data.房屋信息.map(lambda x : x.split('|')[3])
data['楼层'] = data.房屋信息.map(lambda x : x.split('|')[4])
data['建成时间'] = data.房屋信息.map(lambda x : x.split('|')[5]) 
data['结构'] = data.房屋信息.map(lambda x : x.split('|')[6]) 
# 删除房屋信息列
data = data.drop('房屋信息', axis = 1)
data.head()

,描述,位置信息,区域,关注信息,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,135人关注 / 6个月以前发布,91.8万,单价10625元/平米,3室2厅,86.4平米,东,简装,中楼层(共26层),2016年建,塔楼
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,40人关注 / 2个月以前发布,128.5万,单价19435元/平米,3室1厅,66.12平米,东,毛坯,高楼层(共33层),2017年建,板塔结合
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,58人关注 / 15天以前发布,153万,单价20791元/平米,3室1厅,73.59平米,南,精装,中楼层(共34层),2017年建,板塔结合
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,36人关注 / 2个月以前发布,89万,单价11804元/平米,2室1厅,75.4平米,东 北,简装,中楼层(共16层),2011年建,板楼
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,38人关注 / 1个月以前发布,64.5万,单价12479元/平米,1室1厅,51.69平米,南,精装,高楼层(共15层),2007年建,板楼


In [7]:
# 检索每条关注信息记录中保存的信息个数
nrec = data.关注信息.map(lambda x : len(x.split('/')))
nrec.value_counts()

2    39045
Name: 关注信息, dtype: int64

In [8]:
# 拆分关注信息所在列，并将拆分结果增至新标签列
data['关注人数'] = data.关注信息.map(lambda x : x.split('/')[0])
data['发布时间'] = data.关注信息.map(lambda x : x.split('/')[1])
 
# 删除房屋信息列
data = data.drop('关注信息', axis = 1)

data.head()

,描述,位置信息,区域,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构,关注人数,发布时间
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8万,单价10625元/平米,3室2厅,86.4平米,东,简装,中楼层(共26层),2016年建,塔楼,135人关注,6个月以前发布
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5万,单价19435元/平米,3室1厅,66.12平米,东,毛坯,高楼层(共33层),2017年建,板塔结合,40人关注,2个月以前发布
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153万,单价20791元/平米,3室1厅,73.59平米,南,精装,中楼层(共34层),2017年建,板塔结合,58人关注,15天以前发布
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89万,单价11804元/平米,2室1厅,75.4平米,东 北,简装,中楼层(共16层),2011年建,板楼,36人关注,2个月以前发布
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5万,单价12479元/平米,1室1厅,51.69平米,南,精装,高楼层(共15层),2007年建,板楼,38人关注,1个月以前发布


In [9]:
# 缺失值检查
data.isnull()

,描述,位置信息,区域,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构,关注人数,发布时间
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52248,False,False,False,False,False,False,False,False,False,False,False,False,False,False
52249,False,False,False,False,False,False,False,False,False,False,False,False,False,False
52250,False,False,False,False,False,False,False,False,False,False,False,False,False,False
52251,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [10]:
# 统计缺失值个数
(data.isnull()).sum()

描述      0
位置信息    0
区域      0
总价      0
单价      0
户型      0
面积      0
朝向      0
类型      0
楼层      0
建成时间    0
结构      0
关注人数    0
发布时间    0
dtype: int64

In [11]:
# 检查重复值
(data.duplicated()).sum()

16

In [12]:
# 抛弃重复值
data.drop_duplicates(inplace=True)

In [13]:
# 使用正则表达式查看单价列中含有的中文字符种类
data.总价.map(lambda x : re.sub('[^\u4E00-\u9FA5]','',x)).unique()

array(['万'], dtype=object)

In [14]:
# 删去字符“万”，并将类型转换为float，并保留两位小数
data['总价'] = data.总价.map(lambda x : round(float(x.replace('万', '')),2))
data.head(5)

,描述,位置信息,区域,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构,关注人数,发布时间
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8,单价10625元/平米,3室2厅,86.4平米,东,简装,中楼层(共26层),2016年建,塔楼,135人关注,6个月以前发布
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5,单价19435元/平米,3室1厅,66.12平米,东,毛坯,高楼层(共33层),2017年建,板塔结合,40人关注,2个月以前发布
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153.0,单价20791元/平米,3室1厅,73.59平米,南,精装,中楼层(共34层),2017年建,板塔结合,58人关注,15天以前发布
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89.0,单价11804元/平米,2室1厅,75.4平米,东 北,简装,中楼层(共16层),2011年建,板楼,36人关注,2个月以前发布
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5,单价12479元/平米,1室1厅,51.69平米,南,精装,高楼层(共15层),2007年建,板楼,38人关注,1个月以前发布


In [15]:
# 依次检查其他列所含的中文字符
print(data.单价.map(lambda x : re.sub('[^\u4E00-\u9FA5]','',x)).unique())
print(data.面积.map(lambda x : re.sub('[^\u4E00-\u9FA5]','',x)).unique())
print(data.建成时间.map(lambda x : re.sub('[^\u4E00-\u9FA5]','',x)).unique())
print(data.关注人数.map(lambda x : re.sub('[^\u4E00-\u9FA5]','',x)).unique())

['单价元平米']
['平米']
['年建' '板塔结合' '板楼' '暂无数据' '塔楼']
['人关注']


In [16]:
# 建成时间列不包括关键字'年建'的记录数
len(data[~data.建成时间.str.contains('年建')])

245

In [17]:
# 只保留含关键字'年建'的记录
data = data[data.建成时间.str.contains('年建')]

In [18]:
# 将单价列转换为float类型
data['单价'] = data.单价.map(lambda x : round(float(re.findall(r'单价(.*?)元/平米',x)[0])/10000,2))
# 将面积、建成时间和关注人数列转换为float类型
data['面积'] = data.面积.map(lambda x : round(float(x.replace('平米','')),2))
data['建成时间'] = data.建成时间.map(lambda x : float(x.replace('年建','')))
data['关注人数'] = data.关注人数.map(lambda x : float(x.replace('人关注','')))

data.head()

,描述,位置信息,区域,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构,关注人数,发布时间
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8,1.06,3室2厅,86.40,东,简装,中楼层(共26层),2016.0,塔楼,135.0,6个月以前发布
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5,1.94,3室1厅,66.12,东,毛坯,高楼层(共33层),2017.0,板塔结合,40.0,2个月以前发布
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153.0,2.08,3室1厅,73.59,南,精装,中楼层(共34层),2017.0,板塔结合,58.0,15天以前发布
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89.0,1.18,2室1厅,75.40,东 北,简装,中楼层(共16层),2011.0,板楼,36.0,2个月以前发布
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5,1.25,1室1厅,51.69,南,精装,高楼层(共15层),2007.0,板楼,38.0,1个月以前发布


In [19]:
# 查看户型有多少种类
data.户型.unique()

array(['3室2厅 ', '3室1厅 ', '2室1厅 ', '1室1厅 ', '2室2厅 ', '4室2厅 ', '5室2厅 ',
       '4室1厅 ', '1室2厅 ', '3室3厅 ', '1室0厅 ', '5室1厅 ', '6室2厅 ', '4室3厅 ',
       '6室1厅 ', '3室0厅 ', '6室4厅 ', '5室3厅 ', '4室4厅 ', '6室3厅 ', '7室2厅 ',
       '7室3厅 ', '2室0厅 ', '4室0厅 ', '5室0厅 ', '8室2厅 ', '3室4厅 ', '7室1厅 ',
       '7室4厅 ', '9室2厅 ', '8室3厅 ', '5室4厅 ', '0室1厅 ', '7室5厅 ', '0室0厅 '],
      dtype=object)

In [20]:
# 对户型使用独热编码并加入到原有数据帧中
data = data.join(pd.get_dummies(data.户型))
# 删除原有列
data = data.drop('户型', axis = 1)

data.head()

,描述,位置信息,区域,总价,单价,面积,朝向,类型,楼层,建成时间,...,6室3厅,6室4厅,7室1厅,7室2厅,7室3厅,7室4厅,7室5厅,8室2厅,8室3厅,9室2厅
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8,1.06,86.40,东,简装,中楼层(共26层),2016.0,...,0,0,0,0,0,0,0,0,0,0
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5,1.94,66.12,东,毛坯,高楼层(共33层),2017.0,...,0,0,0,0,0,0,0,0,0,0
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153.0,2.08,73.59,南,精装,中楼层(共34层),2017.0,...,0,0,0,0,0,0,0,0,0,0
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89.0,1.18,75.40,东 北,简装,中楼层(共16层),2011.0,...,0,0,0,0,0,0,0,0,0,0
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5,1.25,51.69,南,精装,高楼层(共15层),2007.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# 查看区域有多少种类
print(data.区域.unique())

['双流' '大邑' '天府新区' '天府新区南区' '崇州' '彭州' '成华' '新津' '新都' '武侯' '温江' '简阳' '蒲江'
 '郫都' '都江堰' '金堂' '金牛' '锦江' '青白江' '青羊' '高新' '高新西' '龙泉驿']


In [22]:
# 对区域使用独热编码并加入到原有数据帧中
data = data.join(pd.get_dummies(data.区域))

In [23]:
# 查看类型有多少种类
print(data.类型.unique())
# 查看结构有多少种类
print(data.结构.unique())

[' 简装 ' ' 毛坯 ' ' 精装 ' ' 其他 ']
[' 塔楼' ' 板塔结合' ' 板楼' ' 暂无数据' ' 平房']


In [24]:
# 去掉字符串前后空格
data['类型'] = data.类型.str.strip()
data['结构'] = data.结构.str.strip()

# 查看类型为其他的记录个数
print(len(data[data.类型 == '其他']))
# 查看结构为暂无数据的记录个数
print(len(data[data.结构 == '暂无数据']))

5061
683


In [25]:
# 丢弃无效数据
data = data[(data.类型 != '其他')&(data.结构 != '暂无数据')]

data = data.join(pd.get_dummies(data.类型))
data = data.join(pd.get_dummies(data.结构))
# 删除原有列
data = data.drop('类型', axis = 1)
data = data.drop('结构', axis = 1)
data.head()

,描述,位置信息,区域,总价,单价,面积,朝向,楼层,建成时间,关注人数,...,高新,高新西,龙泉驿,毛坯,简装,精装,塔楼,平房,板塔结合,板楼
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8,1.06,86.40,东,中楼层(共26层),2016.0,135.0,...,0,0,0,0,1,0,1,0,0,0
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5,1.94,66.12,东,高楼层(共33层),2017.0,40.0,...,0,0,0,1,0,0,0,0,1,0
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153.0,2.08,73.59,南,中楼层(共34层),2017.0,58.0,...,0,0,0,0,0,1,0,0,1,0
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89.0,1.18,75.40,东 北,中楼层(共16层),2011.0,36.0,...,0,0,0,0,1,0,0,0,0,1
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5,1.25,51.69,南,高楼层(共15层),2007.0,38.0,...,0,0,0,0,0,1,0,0,0,1


In [26]:
# 查看朝向列种类
data.朝向.unique()

array([' 东 ', ' 南 ', ' 东 北 ', ' 西北 ', ' 西 ', ' 南 北 ', ' 东 西 ', ' 东北 ',
       ' 东南 ', ' 北 ', ' 西南 ', ' 西北 北 ', ' 西南 西 ', ' 东 南 ', ' 南 西南 ',
       ' 东南 南 ', ' 西 西北 ', ' 东 东南 ', ' 东南 西 ', ' 南 西 ', ' 东南 西南 ',
       ' 西南 东北 ', ' 东 西南 ', ' 北 南 ', ' 东 东南 南 ', ' 西 西南 ', ' 西 北 ',
       ' 西南 北 ', ' 北 东北 ', ' 东南 北 ', ' 东南 东北 ', ' 西南 南 东南 ', ' 东 北 西南 ',
       ' 东 西北 ', ' 东 东南 西 ', ' 南 东南 ', ' 西南 南 ', ' 东 东南 东北 ', ' 东南 西北 ',
       ' 东 北 西 ', ' 东南 南 东 ', ' 西南 西北 ', ' 西 南 ', ' 西 东 ', ' 东 南 西 ',
       ' 西南 东 ', ' 东南 南 西南 ', ' 东 东北 ', ' 东南 东 ', ' 北 西北 ', ' 南 西北 ',
       ' 南 东北 ', ' 东北 南 ', ' 东北 东南 ', ' 东南 南 北 ', ' 南 西南 东北 ', ' 西南 西 南 ',
       ' 南 东 ', ' 西北 西 ', ' 南 西北 北 ', ' 东 南 西 北 ', ' 西 东北 ',
       ' 东 东南 西 西北 ', ' 东北 西南 ', ' 东 南 北 ', ' 东北 东南 西南 ', ' 南 北 东北 ',
       ' 西北 东南 ', ' 西北 东北 ', ' 西北 西南 ', ' 西南 东南 ', ' 南 西南 西 ', ' 北 东南 ',
       ' 东南 西 西北 ', ' 西 西南 西北 ', ' 西南 南 东南 东 ', ' 东 东南 西南 西 ', ' 东 西 北 ',
       ' 东北 东 ', ' 南 东南 西南 ', ' 北 东 ', ' 东 西 南 北 ', ' 东南 西南 西北 ',
       ' 东北 西北 ', ' 

In [27]:
def my_get_dummies(ser):
    base_dirt = ['东', '南', '西', '北', '东北', '东南', '西南', '西北']
    base_data = np.zeros((len(ser),), dtype=np.int)
    df = pd.DataFrame({'东':base_data,'南':base_data, \
                       '西':base_data, '北':base_data, \
                       '东北':base_data, '东南':base_data, \
                       '西南':base_data, '西北':base_data}, index=ser.index)
   
    for irec in ser.index:
        # 分隔字符串
        rec = ser[irec].strip().split(' ')
        # 遍历每条记录分隔后的方位
        for dirt in rec:
            # 检查是否存在8个基本方位以外的记录
            if dirt not in base_dirt:
                print(dirt)
            else:
                df[dirt][irec] = 1
                
    return df

In [28]:
# 自定义独热编码
data = data.join(my_get_dummies(data.朝向))
# 删除原有列
data = data.drop('朝向', axis = 1)
data.head()

C:\Users\yaojie\AppData\Local\Temp\ipykernel_14624\3416521185.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  base_data = np.zeros((len(ser),), dtype=np.int)


,描述,位置信息,区域,总价,单价,面积,楼层,建成时间,关注人数,发布时间,...,板塔结合,板楼,东,南,西,北,东北,东南,西南,西北
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8,1.06,86.40,中楼层(共26层),2016.0,135.0,6个月以前发布,...,0,0,1,0,0,0,0,0,0,0
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5,1.94,66.12,高楼层(共33层),2017.0,40.0,2个月以前发布,...,1,0,1,0,0,0,0,0,0,0
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153.0,2.08,73.59,中楼层(共34层),2017.0,58.0,15天以前发布,...,1,0,0,1,0,0,0,0,0,0
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89.0,1.18,75.40,中楼层(共16层),2011.0,36.0,2个月以前发布,...,0,1,1,0,0,1,0,0,0,0
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5,1.25,51.69,高楼层(共15层),2007.0,38.0,1个月以前发布,...,0,1,0,1,0,0,0,0,0,0


In [29]:
# 检测数据格式一致性
(~data.楼层.str.contains('楼层')).sum()

453

In [30]:
# 舍弃数据
data = data[data.楼层.str.contains('楼层')]
# 查看数据唯一值
data.楼层.unique()

array([' 中楼层(共26层) ', ' 高楼层(共33层) ', ' 中楼层(共34层) ', ' 中楼层(共16层) ',
       ' 高楼层(共15层) ', ' 低楼层(共32层) ', ' 中楼层(共32层) ', ' 中楼层(共14层) ',
       ' 高楼层(共23层) ', ' 中楼层(共33层) ', ' 高楼层(共32层) ', ' 中楼层(共13层) ',
       ' 中楼层(共15层) ', ' 高楼层(共6层) ', ' 低楼层(共12层) ', ' 高楼层(共31层) ',
       ' 中楼层(共5层) ', ' 中楼层(共18层) ', ' 高楼层(共16层) ', ' 中楼层(共25层) ',
       ' 低楼层(共18层) ', ' 低楼层(共13层) ', ' 中楼层(共12层) ', ' 高楼层(共14层) ',
       ' 低楼层(共6层) ', ' 高楼层(共18层) ', ' 高楼层(共34层) ', ' 高楼层(共13层) ',
       ' 中楼层(共23层) ', ' 中楼层(共17层) ', ' 高楼层(共12层) ', ' 低楼层(共26层) ',
       ' 低楼层(共17层) ', ' 中楼层(共24层) ', ' 高楼层(共7层) ', ' 中楼层(共22层) ',
       ' 中楼层(共6层) ', ' 高楼层(共24层) ', ' 低楼层(共14层) ', ' 中楼层(共11层) ',
       ' 中楼层(共30层) ', ' 高楼层(共38层) ', ' 高楼层(共26层) ', ' 低楼层(共9层) ',
       ' 高楼层(共17层) ', ' 低楼层(共29层) ', ' 低楼层(共27层) ', ' 低楼层(共33层) ',
       ' 低楼层(共16层) ', ' 低楼层(共22层) ', ' 高楼层(共29层) ', ' 低楼层(共25层) ',
       ' 中楼层(共7层) ', ' 低楼层(共11层) ', ' 低楼层(共23层) ', ' 高楼层(共9层) ',
       ' 中楼层(共29层) ', ' 低楼层(共20层) ', ' 低楼层(共15层) ', ' 低楼层(共34层) ',
   

In [31]:
# 提取所在楼层
data['所在楼层'] = data.楼层.map(lambda x : x.split('(')[0])

# 对所在楼层进行独热编码
data = data.join(pd.get_dummies(data.所在楼层))

# 使用正则表达式提取数据并转换为int类型
data['总楼层'] = data.楼层.map(lambda x : int(re.findall(r'\(共(.*?)层\)', x)[0]))

# 删除原有列
data = data.drop('楼层', axis = 1)
data = data.drop('所在楼层', axis = 1)

data.head()

,描述,位置信息,区域,总价,单价,面积,建成时间,关注人数,发布时间,0室0厅,...,西,北,东北,东南,西南,西北,中楼层,低楼层,高楼层,总楼层
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8,1.06,86.40,2016.0,135.0,6个月以前发布,0,...,0,0,0,0,0,0,1,0,0,26
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5,1.94,66.12,2017.0,40.0,2个月以前发布,0,...,0,0,0,0,0,0,0,0,1,33
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153.0,2.08,73.59,2017.0,58.0,15天以前发布,0,...,0,0,0,0,0,0,1,0,0,34
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89.0,1.18,75.40,2011.0,36.0,2个月以前发布,0,...,0,1,0,0,0,0,1,0,0,16
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5,1.25,51.69,2007.0,38.0,1个月以前发布,0,...,0,0,0,0,0,0,0,0,1,15


In [32]:
# 删除发布时间信息
data = data.drop('发布时间', axis = 1)
# 去掉列标题空白
data = data.rename(columns = lambda x:x.strip())
# 保存数据
output_file_path = '房产信息_预处理.xlsx'
data.to_excel(output_file_path, index=False)